# Sprint_1 機械学習フロー

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.1.3
Scikit-Learn 0.23.2
GPU is available


【事前工程】
1. 事前学習で作成したベースラインモデルを展開する。 
    1. データの前処理
        1. csvファイルの読み出し
        2. objectをlabel/one-hot変換して数値化
        3. 異常値の処置
        4. 欠損値の処置
    2. 説明変数を選抜
        1. 相関係数を算出
        2. 高い相関性を4つ選択
2. hold-out法ではなく、cross validationを行う。
    1. scikit-learnのkFoldクラスを用いる
    2. cross_val_scoreにて精度を確認する
3. それ以降（学習・推定）は行わない。

In [26]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
# import seaborn as sns

### データの前処理（csvファイルの読み出し）

In [27]:
pwd

'C:\\Users\\Nacho\\Documents\\Coding\\DIC\\Sprint'

In [28]:
# csvファイルの読み出し

app_train = pd.read_csv('../Data/home_credit_defalut_risk_application_train.csv')
app_test = pd.read_csv('../Data/home_credit_defalut_risk_application_test.csv')

print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### データの前処理（object型をlabel/one-hot変換してint化）

In [29]:
# objectをlabel/one-hot変換して数値化

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [30]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)
Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)


### データの前処理（異常値の排除）

In [31]:
# Skipping anomalies

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

There are 9274 anomalies in the test data out of 48744 entries


### データの前処理（欠損値の処置）

In [32]:
# imputer for handling missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy = 'median')

X = np.array(app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']])
y = np.array(app_train[['TARGET']])

X = imputer.fit_transform(X)
pd_X = pd.DataFrame(X)
pd_X

,0,1,2,3
0,0.083037,0.262949,0.139376,-9461.0
1,0.311267,0.622246,0.535276,-16765.0
2,0.505998,0.555912,0.729567,-19046.0
3,0.505998,0.650442,0.535276,-19005.0
4,0.505998,0.322738,0.535276,-19932.0
...,...,...,...,...
307506,0.145570,0.681632,0.535276,-9327.0
307507,0.505998,0.115992,0.535276,-20775.0
307508,0.744026,0.535722,0.218859,-14966.0
307509,0.505998,0.514163,0.661024,-11961.0


In [33]:
# 欠損値の確認
# print(pd_X.isnull().any())
# Xとyデータの確認
print('Training data shape: ', X.shape)
print('Testing data shape: ', y.shape)

Training data shape:  (307511, 4)
Testing data shape:  (307511, 1)


## 【問題1】クロスバリデーション 

「事前学習期間の課題で作成したベースラインモデルに対応したKFoldクラスによる訓練と交差検証（クロスバリデーション）で評価を行うパイプラインを作成する。」

-----
Point
* ホールとアウト法ではなく、クロスバリデーション（交差検証法）を行う。
    * ホールドアウト法：学習用と評価用で２分割
    * クロスバリデーション（交差検証法）：学習用と評価用で２分割を１通り以上作成する。
-----
Workflow
* scikit-learnのkFoldクラスを用いる。
    * n_splits = 10回分割する。
    * random_state = 1で固定
    * 説明変数：X
    * 目的変数：y
* 交差検証をパイプラインを用いて検証する。

In [21]:
# ライブラリをロード
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [22]:
# 【参考】以下がhold-out法である。
# 各Xとyのtrainとtestにsplitする。
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [     6     19     20 ... 307488 307493 307495]
TRAIN: [     0      2      3 ... 307508 307509 307510] TEST: [     1      4      8 ... 307471 307480 307501]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [     3     18     30 ... 307474 307499 307503]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [    22     29     38 ... 307473 307487 307504]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [    21     42     56 ... 307475 307489 307502]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [     9     10     15 ... 307483 307486 307492]
TRAIN: [     0      1      2 ... 307507 307508 307509] TEST: [    12     16     25 ... 307484 307500 307510]
TRAIN: [     0      1      2 ... 307507 307508 307510] TEST: [    23     37     55 ... 307497 307505 307509]
TRAIN: [     0      1      3 ... 307506 307509 307510] TEST: [     2     11     35 ... 307494 307507 307508]
TRAIN: [     1     

In [18]:
# 標準化器を作成
standardizer = StandardScaler()

# ロジスティック回帰器を作成
logit = LogisticRegression()

# 標準化とロジスティック回帰を同時に行うパイプラインを作成
pipeline = make_pipeline(standardizer, logit)

# k-分割交差検証器を作成
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# k-分割交差検証を実行
cv_results = cross_val_score(pipeline, #上記で定義したパイプライン
                            X, # 特徴量行列
                            y, # ターゲットベクトル
                            cv=kf, # 交差検証手法
                            scoring="accuracy", # スコア関数
                            n_jobs=-1) # 全てのCPUを利用

# 平均値を計算
cv_results.mean()

0.919134596280229

## 【問題2】グリッドサーチ

1. scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。
2. ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。
-----
* グリッドサーチとは全てのパラメータの組み合わせを全て試して、最も評価精度の良いモノを探索する手法。
-----
* ベースラインモデル
    * 説明変数：X
    * 目的変数：y
    * モデル：ロジスティック回帰
* パラメータの調整対象
    * C
    * 正則化ペナルティ

### GridSearchCVを行うコード

In [34]:
# ライブラリーをロード
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# ロジスティック回帰器を作成
logistic = linear_model.LogisticRegression()

# 正則化強度ハイパーパラメータの候補となる値の範囲を指定
C = np.logspace(0, 4, 10)

# 正則化ペナルティハイパーパラメータの候補となる値の範囲を指定
penalty = ['l1', 'l2']

# ハイパーパラメータの候補辞書を作成。
hyperparameters = dict(C=C, penalty=penalty)

# グリッド探索器を作成
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

# グリッド探索器を訓練
best_model = gridsearch.fit(X, y)

print("Best Penalty:", best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l2
Best C: 1.0


### ベースラインモデルへのパラメーターの適応

* 正則化ペナルティハイパーパラメータ: l2
* 正則化強度ハイパーパラメータ: 1.0

In [35]:
# 標準化器を作成
standardizer = StandardScaler()

# ロジスティック回帰器を作成（ハイパーパラメータをチューニング）
logit = LogisticRegression(C=0.09, penalty='l1')

# 標準化とロジスティック回帰を同時に行うパイプラインを作成
pipeline = make_pipeline(standardizer, logit)

# k-分割交差検証器を作成
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# k-分割交差検証を実行
cv_results = cross_val_score(pipeline, #上記で定義したパイプライン
                            X, # 特徴量行列
                            y, # ターゲットベクトル
                            cv=kf, # 交差検証手法
                            scoring="accuracy", # スコア関数
                            n_jobs=-1) # 全てのCPUを利用

# 平均値を計算
cv_results

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [36]:
# GridSearchCVによる再学習
best_model.predict(X)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## 【問題3】Kaggle Notebooksからの調査

KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

* 異常値の処置（適応済み）
    * Anomalies（異常値）は原因は様々
    * 最も安全な処置としては、欠損値として扱い、後の欠損値の処置に廻す。（平均値で補完する）
* object型のint化（適応済み）
    * 二値の場合はlabel encoding、二値以上の場合はone-hot encodingでカラム数を増やす。
* 欠損値の処置（適応済み）
    * SimpleImputerを用い、欠損値(NaN）に平均値を代入して補完する。
* 相関係数行列（適応済み）
    * 欠損値の処置の前に行う。
* polynomical_features (多項式回帰）因果変数同士の組み合わせ 
    * 結果（目的変数）に対して、個々の説明変数の因果関係は乏しくとも、組み合わせにより相関性が向上する。
* Light Gradient Boosting Machine (LightGBM)
    * L・G・B・M！
* 複数の学習アルゴリズムから最良のモデルを選択
    * 一度のグリッド探索によって、ロジスティック回帰、ランダムフォレスト等の複数のモデルを比較検証する。
    * 同時に上記複数のモデルから最良のハイパーパラメータを候補範囲から選択できる。

## 【問題4】高い汎化性能のモデル作成

* グリッド探索器を用いて、複数のモデルとパラメータを比較検証する。
    * ロジスティック回帰とランダムフォレスト回帰を用いたモデルの精度
* ~~LightGBMを適応したモデルの精度~~

### グリッド探索器を作成

In [27]:
# ライブラリーをロード
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# パイプラインを作成
pipe = Pipeline([("classifier", RandomForestClassifier())])

# 候補学習アルゴリズムとそのハイパーパラメータの辞書を作成
search_space = [{"classifier": [LogisticRegression()],
                "classifier__penalty": ['l1', 'l2'],
                "classifier__C": np.logspace(0, 4, 10)},
               {"classifier": [RandomForestClassifier()],
               "classifier__n_estimators": [10, 100, 1000],
               "classifier__max_features": [1, 2, 3]}]

# グリッド探索器を作成
gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=3, n_jobs=-1)

# グリッド探索器を訓練
best_model = gridsearch.fit(X, y)

Fitting 5 folds for each of 29 candidates, totalling 145 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 145 out of 145 | elapsed: 35.8min finished


In [30]:
best_model.best_estimator_.get_params()

{'memory': None,
 'steps': [('classifier', LogisticRegression())],
 'verbose': False,
 'classifier': LogisticRegression(),
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__dual': False,
 'classifier__fit_intercept': True,
 'classifier__intercept_scaling': 1,
 'classifier__l1_ratio': None,
 'classifier__max_iter': 100,
 'classifier__multi_class': 'auto',
 'classifier__n_jobs': None,
 'classifier__penalty': 'l2',
 'classifier__random_state': None,
 'classifier__solver': 'lbfgs',
 'classifier__tol': 0.0001,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

In [41]:
# ロジスティック回帰器を作成（ハイパーパラメータをチューニング）
logit = LogisticRegression()

# 正則化強度ハイパーパラメータを指定
C = np.logspace(0, 4, 10)

# 正則化ペナルティハイパーパラメータを指定
penalty = ['l2']

# ハイパーパラメータの辞書を作成。
hyperparameters = dict(C=C, penalty=penalty)

#グリッド探索器を作成
gridsearch = GridSearchCV(logit, hyperparameters, cv=5, n_jobs=-1, verbose=0)

# 二重交差検証を行い、平均値を表示
cross_val_score(gridsearch, X, y).mean()

0.9192711805397202

### 結論

1. ロジスティック回帰（l1）を適応したcross_val_score：nan
2. ロジスティック回帰（l2）を適応したcross_val_score：0.919

ランダムフォレストとロジスティック回帰ではGridSearchCVはロジスティック回帰を選択した。
パラメータ調整でも、同じロジスティック回帰でも正則化L1（ラッソ回帰）と正則化L2（リッジ回帰）とで、異なる結果が出た。

In [45]:
# pandasで表を作成

list = [['nan'], [0.919]]
df = pd.DataFrame(list)
df.columns = ['cross_val_score']
df.index = ['LogisticRegression [l1]', 'LogisticRegression [l2]']

df

,cross_val_score
LogisticRegression [l1],nan
LogisticRegression [l2],0.919


## 【問題5】最終的なモデルの選定

1. 最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認する。
2. どういったアイデアを取り入れ、どの程度のスコアになったかを記載する。
-----
【解答】
* ランダムフォレストとロジスティック回帰とで比較した場合はロジスティック回帰が好ましいと選択された。
* ロジスティック回帰でのハイパーパラメータのチューニングでもC=1.0、ペナルティはL1（ラッソ）が良い結果が出ると判明した。
    * そもそもL2はnan結果のため、精度の数値比較が乏しい結果であるが。
* 標準化器を用いると、数値がnanになる場合がある。
    * 原因は不明

In [ ]:
# RandomForestRegressorのみをKaggleに提出
gridsearch = GridSearchCV(logit, hyperparameters, cv=5, verbose=0, n_jobs=-1)
Gridsearch.fit(gridsearch)

# Make predictions on the test data
predictions = GridSearch.predict_proba(X)

# Make a submission dataframe
submit = X[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline.csv', index = False)